In [1]:
import pandas as pd
import numpy as np

In [2]:
file_path = 'CSV Rotas - Base de Dados.csv'

with open(file_path, 'r') as file:
    lines = file.readlines()
    cleaned_lines = [line.strip(',\n') for line in lines]

from io import StringIO
cleaned_data = StringIO('\n'.join(cleaned_lines))
distancias_df = pd.read_csv(cleaned_data, index_col=0)
distancias_df = distancias_df.apply(pd.to_numeric, errors='coerce').fillna(float('inf'))

print("Dados limpos:")
print(distancias_df)

Dados limpos:
                      Faminas  Bar do Lu  Bar da fatinha  Praca do rosario  \
Joao pinheiro             460        220             550                60   
Faminas                     0        430             350               480   
Mansao gastropub          350        190             300               240   
Bar da fatinha            350        260               0               430   
Bar do broa               520        300             580                85   
Rei do cachorao           510        250             460                20   
Lagoa da Gavea            660        170             280               180   
Cristo                    630        170             330               110   
Bar betinho               800        270             480                50   
Santa marcelina           680        170             360               120   
Hospital Sao Paulo        780        250             460               130   
Praca do rosario          750        220          

In [3]:
locais_df = pd.read_csv('locais.csv')
locais = locais_df['Locais'].tolist()

print("Locais do arquivo locais.csv:", locais)
print("Cidades do DataFrame de distâncias:", list(distancias_df.index))

Locais do arquivo locais.csv: ['Faminas', 'Joao pinheiro', 'Dule']
Cidades do DataFrame de distâncias: ['Joao pinheiro', 'Faminas', 'Mansao gastropub', 'Bar da fatinha', 'Bar do broa', 'Rei do cachorao', 'Lagoa da Gavea', 'Cristo', 'Bar betinho', 'Santa marcelina', 'Hospital Sao Paulo', 'Praca do rosario', 'Inove', 'Dule', 'Armacao', 'Bar do Lu', 'Fest house', 'Cachorao do naldinho']


In [4]:
cidades_comuns = list(set(distancias_df.index).intersection(locais))
distancias_df = distancias_df.loc[cidades_comuns, cidades_comuns]
locais_finais = [local for local in locais if local in cidades_comuns]

if not locais_finais:
    raise ValueError("Nenhum dos locais do arquivo locais.csv está presente na matriz de distâncias.")
print("Cidades finais após o filtro:", cidades_comuns)
print("Locais finais após o filtro:", locais_finais)

Cidades finais após o filtro: ['Dule', 'Faminas', 'Joao pinheiro']
Locais finais após o filtro: ['Faminas', 'Joao pinheiro', 'Dule']


In [5]:
distancias = distancias_df.values
cidades = list(distancias_df.index)

num_formigas = 10
num_iteracoes = 1000
alpha = 1.0
beta = 2.0
rho = 0.5 
Q = 100

num_cidades = len(cidades)
feromonios = np.ones((num_cidades, num_cidades)) / num_cidades

In [6]:
def avalia_solucao(solucao):
    distancia_total = 0
    for i in range(len(solucao) - 1):
        distancia_total += distancias[cidades.index(solucao[i]), cidades.index(solucao[i + 1])]
    distancia_total += distancias[cidades.index(solucao[-1]), cidades.index(solucao[0])]
    return distancia_total

def construir_solucao():
    solucao = []
    visitadas = [False] * num_cidades
    cidade_atual = np.random.choice(locais_finais)
    solucao.append(cidade_atual)
    visitadas[cidades.index(cidade_atual)] = True

    for _ in range(num_cidades - 1):
        probabilidades = []
        for cidade in range(num_cidades):
            if not visitadas[cidade]:
                tau = feromonios[cidades.index(cidade_atual)][cidade] ** alpha
                eta = (1 / distancias[cidades.index(cidade_atual)][cidade]) ** beta if distancias[cidades.index(cidade_atual)][cidade] > 0 else 0
                probabilidades.append(tau * eta)
            else:
                probabilidades.append(0)
        
        probabilidades = np.array(probabilidades)
        soma_probabilidades = probabilidades.sum()
        
        if soma_probabilidades == 0:
            cidades_nao_visitadas = [cidades[cidade] for cidade in range(num_cidades) if not visitadas[cidade]]
            cidade_atual = np.random.choice(cidades_nao_visitadas)
        else:
            probabilidades /= soma_probabilidades
            cidade_atual = np.random.choice(cidades, p=probabilidades)
        
        solucao.append(cidade_atual)
        visitadas[cidades.index(cidade_atual)] = True

    return solucao

In [7]:
melhor_solucao = None
menor_distancia = float('inf')

for iteracao in range(num_iteracoes):
    solucoes = []
    distancias_solucoes = []

    for formiga in range(num_formigas):
        solucao = construir_solucao()
        distancia = avalia_solucao(solucao)
        solucoes.append(solucao)
        distancias_solucoes.append(distancia)

        if distancia < menor_distancia:
            melhor_solucao = solucao
            menor_distancia = distancia

    feromonios *= (1 - rho)
    for solucao, distancia in zip(solucoes, distancias_solucoes):
        for i in range(len(solucao) - 1):
            feromonios[cidades.index(solucao[i])][cidades.index(solucao[i + 1])] += Q / distancia
            feromonios[cidades.index(solucao[i + 1])][cidades.index(solucao[i])] += Q / distancia
        feromonios[cidades.index(solucao[-1])][cidades.index(solucao[0])] += Q / distancia
        feromonios[cidades.index(solucao[0])][cidades.index(solucao[-1])] += Q / distancia

In [8]:
distancia_melhor_solucao = avalia_solucao(melhor_solucao)

print("Melhor solução encontrada:", melhor_solucao)
print("Menor distância encontrada:", menor_distancia)
print("Distância da melhor solução calculada na ordem:", distancia_melhor_solucao, " metros")

Melhor solução encontrada: ['Dule', 'Joao pinheiro', 'Faminas']
Menor distância encontrada: 940
Distância da melhor solução calculada na ordem: 940  metros
